В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.


Таблица city — это справочник городов. Структура справочника представлена ниже.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
city_id	        integer	    уникальный идентификатор города, первичный ключ
city_name	    text	    название города
state	        text	    штат, к которому относится город
population	    integer	    население города
area	        numeric	    площадь города


Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
cust_id	        integer 	уникальный идентификатор клиента, первичный ключ
cust_name	    text	    название клиента
annual_revenue	numeric	    ежегодная выручка
cust_type	    text	    тип пользователя
address	        text	    адрес
zip	            integer	    почтовый индекс
phone	        text	    телефон
city_id	integer	идентификатор города, внешний ключ к таблице city


Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ 	ОПИСАНИЕ
driver_id	    integer	    уникальный идентификатор водителя, первичный ключ
first_name	    text	    имя водителя
last_name   	text	    фамилия водителя
address	        text	    адрес водителя
zip_code	    integer	    почтовый индекс водителя
phone	        text	    телефон водителя
city_id	        integer	    идентификатор города водителя, внешний ключ к таблице city


В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
truck_id	    integer	    Уникальный идентификатор грузовика, первичный ключ
make	        text	    Производитель грузовика
model_year	    integer 	Дата выпуска грузовика


Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
ship_id	        integer 	уникальный идентификатор доставки, первичный ключ
cust_id     	integer	    идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
weight	        numeric	    вес посылки
truck_id	    integer 	идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck
driver_id	    integer	    идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
city_id	        integer 	идентификатор города в который совершена доставка, внешний ключ к таблице city
ship_date	    date	    дата доставки

![Alt text](image.png)

 Укажите название города с максимальным весом единичной доставки.

 SELECT 
    c.city_name
--    s.weight
FROM
    sql.shipment s
    JOIN sql.city c ON s.city_id = c.city_id
ORDER BY s.weight DESC

второй вариант (с весом)

SELECT 
    c.city_name,
    MAX(s.weight)
FROM
    sql.shipment s
    JOIN sql.city c ON s.city_id = c.city_id
GROUP BY c.city_id, s.weight
ORDER BY s.weight DESC

 Сколько различных производителей грузовиков перечислено в таблице truck?

 SELECT 
    distinct
    make
FROM
    sql.truck 

Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

SELECT 
    d.first_name,
    COUNT(s.cust_id)
FROM
    sql.driver d 
    JOIN sql.shipment s ON s.driver_id = d.driver_id
GROUP BY d.first_name, s.cust_id
ORDER BY COUNT(s.cust_id) DESC

Укажите даты первой и последней по времени доставок в таблице shipment.
Ответ введите в формате ДД.ММ.ГГГГ.

SELECT 
    max(ship_date),
    min(ship_date)
FROM
    sql.shipment 

Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

SELECT 
    cu.cust_name,
    COUNT(s.ship_id)
FROM
    sql.customer cu 
    JOIN sql.shipment s ON s.cust_id = cu.cust_id
WHERE s.ship_date >= '2017.01.01' AND s.ship_date < '2018.01.01'
GROUP BY cu.cust_name

ORDER BY COUNT(s.ship_id) DESC

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

ЗапросДетализация
SELECT          book_name object_name, 'книга' object_description 
FROM          public.books
UNION ALL
SELECT          movie_title, 'фильм' 
FROM          sql.kinopoisk
Визуально произведённое нами действие можно представить следующим образом:

![Alt text](image-1.png)

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:

одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.
ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

UNION выводит только уникальные записи;
UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.
Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.
Проверить, как различаются операторы, вы сможете при выполнении заданий 2.1 и 3.1.

СИНТАКСИС

Запрос строится таким образом:

SELECT         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n
Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.
Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL
SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL
SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL
SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALL
SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
Вместо результата получим сообщение о синтаксической ошибке: "...syntax error at or near "UNION"..." Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)
Отлично! Мы получили именно то, что хотели.

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

так много повторяющихся значений

SELECT
         c.city_name object_name, 'city' object_type
FROM 
         sql.city c
UNION ALL
SELECT
         c.state, 'state'
FROM 
         sql.city c
UNION ALL
SELECT
         d.first_name, 'driver'
FROM 
         sql.driver d

UNION ALL
SELECT
         t.make, 'truck'
FROM 
         sql.truck t
ORDER BY 1, 2

убираем ALL:

SELECT
         c.city_name object_name, 'city' object_type
FROM 
         sql.city c
UNION
SELECT
         c.state, 'state'
FROM 
         sql.city c
UNION
SELECT
         d.first_name, 'driver'
FROM 
         sql.driver d

UNION
SELECT
         t.make, 'truck'
FROM 
         sql.truck t
ORDER BY 1, 2

приняли этот вариант, хотя ИИ "подсказывал" вставить ALL

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT
         c.city_name object_name
FROM 
         sql.city c
UNION ALL
SELECT
         c.state
FROM 
         sql.city c
ORDER BY 1

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT
         c.city_name object_name
FROM 
         sql.city c
UNION
SELECT
         c.state
FROM 
         sql.city c
ORDER BY 1

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели column_name::column_type.

Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

SELECT 
         c.city_id::text
FROM
         sql.city c
UNION ALL
SELECT 
         cc.city_name
FROM
         sql.city cc

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

SELECT 
         d.zip_code::text contact,
         d.first_name,
         'zip' contact_type
FROM
         sql.driver d
UNION ALL
SELECT 
         d.phone,
         d.first_name,
         'phone' contact_type
FROM
         sql.driver d

Почему-то прокатило без сортировки

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.


SELECT
         c.city_name,
         c.population /*выбираем столбцы city_name, population*/
FROM
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT
         'total',
         SUM(c.population) /*сами задаём объект ‘total’, суммируем все значения столбца population*/
FROM
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/
ORDER BY 2 DESC /*сортируем по второму столбцу в убывающем порядке (чтобы итоговая сумма была в начале)*/

![Alt text](image-2.png)

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.

SELECT 
    s.ship_date::text date_period, 
    COUNT(s.ship_id) cnt_shipment
FROM
         sql.shipment s
GROUP BY s.ship_date
UNION ALL
SELECT 
         'total_shipments', COUNT(s.ship_id)
FROM
         sql.shipment s
ORDER BY 1 DESC

Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.


SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NOT NULL

UNION

SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NULL

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:

если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
если нет — выводим 'доставка не осуществлялась'.
Столбцы к выводу: city_name, state, shipping_status. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

SELECT
        c.city_name,
        c.state,
        'доставка осуществлялась' shipping_status
FROM
        sql.city c
        LEFT JOIN sql.shipment s ON c.city_id = s.city_id
WHERE s.ship_id IS NOT NULL

UNION

SELECT
        c.city_name,
        c.state,
        'доставка не осуществлялась' shipping_status
FROM
        sql.city c
        LEFT JOIN sql.shipment s ON c.city_id = s.city_id
WHERE s.ship_id IS NULL

ORDER BY 1, 2

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:

если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

SELECT
        c.city_name,
        COUNT(s.ship_id) shippings_fake
FROM
        sql.city c
        LEFT JOIN sql.shipment s ON c.city_id = s.city_id
GROUP BY c.city_name, s.city_id
HAVING COUNT(s.ship_id) > 10

UNION ALL

SELECT
        c.city_name,
        COUNT(s.ship_id) + 5
FROM
        sql.city c
        LEFT JOIN sql.shipment s ON c.city_id = s.city_id
GROUP BY c.city_name, s.city_id
HAVING COUNT(s.ship_id) <= 10 AND COUNT(s.ship_id) > 0

ORDER BY 2 DESC, 1

Union чаще всего используется для создания "синтетических", искуственных таблиц

SELECT 
         'a' letter,'1' ordinal_position
UNION 
SELECT 
         'b','2'
UNION 
SELECT
         'c','3'

Напишите запрос, который выберет наибольшее из значений:

1000000;
541;
-500;
100.
Столбец с результатом назовите result.


SELECT 1000000 result
UNION ALL
SELECT 541
UNION ALL 
SELECT -500
UNION ALL
SELECT 100
ORDER BY 1 DESC
LIMIT 1

Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

1000000;
541;
-500;
100.
Столбец с ответом назовите mycol

SELECT 1000000::text mycol
UNION ALL
SELECT 541::text
UNION ALL 
SELECT (-500)::text
UNION ALL
SELECT 100::text
ORDER BY 1 DESC
LIMIT 1


Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

+ ;
- ;
= ;
/ .
Столбец с ответом назовите result.


SELECT '+' mycol
UNION ALL
SELECT '-'::text
UNION ALL 
SELECT '='::text
UNION ALL
SELECT '/'::text
ORDER BY 1 DESC
LIMIT 1

SELECT
         c.city_name
FROM
         sql.shipment s
JOIN sql.city c ON s.city_id = c.city_id
EXCEPT
SELECT
         cc.city_name
FROM
         sql.driver d 
JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1

Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip. В поле ниже введите запрос, с помощью которого вы решили эту задачу.


SELECT
        d.zip_code zip
FROM
        sql.driver d
--JOIN sql.city c ON s.city_id = c.city_id
EXCEPT
SELECT
        c.zip
FROM
        sql.customer c 
--JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1

SELECT          c.city_name object_name
FROM          sql.city c
INTERSECT
SELECT 
         cc.state
FROM          sql.city cc
ORDER BY 1

Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.


SELECT c.city_id id
FROM sql.customer c
INTERSECT
SELECT d.city_id
FROM sql.driver d
INTERSECT
SELECT s.city_id
FROM sql.shipment s
ORDER BY 1

Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.


SELECT c.zip zip
FROM sql.customer c
INTERSECT
SELECT d.zip_code
FROM sql.driver d
--INTERSECT
--SELECT s.city_id
--FROM sql.shipment s
ORDER BY 1

## <center> ИТОГИ </center>

SELECT          N columns
FROM          table_1
UNION / UNION ALL / EXCEPT / INTERSECT 
SELECT          N columns
FROM          table_2

Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.


SELECT 
    c.city_name,
    s.weight weight
FROM
    sql.shipment s
    JOIN sql.city c ON s.city_id = c.city_id
ORDER BY s.weight DESC
LIMIT 1

UNION

SELECT 
    c.city_name,
    s.weight weight
FROM
    sql.shipment s
    JOIN sql.city c ON s.city_id = c.city_id
ORDER BY s.weight
LIMIT 1

Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id). Столбец к выводу — mutual_id. Отсортируйте по возрастанию.


SELECT 
    s.cust_id mutual_id
FROM
    sql.shipment s

INTERSECT

SELECT 
    s.ship_id
FROM
    sql.shipment s
ORDER BY 1

Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.


SELECT 
    c.cust_name object_name,
    'КЛИЕНТ' object_description
FROM
    sql.customer c
WHERE cust_type = 'manufacturer'

UNION

SELECT 
    t.make ,
    'ГРУЗОВИК' 
FROM
    sql.truck t

ORDER BY 1

Знания по темам этого модуля зачастую проверяют работодатели на собеседованиях. Предлагаем вам ознакомиться с вопросами, с которыми вы можете столкнуться при поиске работы. 

Важно! Это задания на самопроверку, они необязательные и мы не будем оценивать качество их решения. На некоторые вопросы будет предоставляться ответ, а на некоторые — нет. Настоятельно рекомендуем вам попробовать найти ответ самостоятельно, прежде чем открывать его.

⚡ Мы не добавляем скрипты для таблиц, которые нужны для заданий, так как именно в таком виде их получают кандидаты на собеседованиях.

Дан источник данных — таблица TEST_SQL
![Alt text](image-3.png)

Поля таблицы:

ST_ORD (Varchar) — статус заявки;
TYPE_PRODUCT_NAME (Varchar) — тип заявки;
PRODUCT_INFOSOURCE1 (Varchar) — источник;
CREATE_DATE (Date) — дата создания заявки;
INDEX_LEAD (Number) — индикатор заявки (флаг 0/1 определяет регистрацию лида в системе);
INDEX_ISSUE (Number) — индикатор выдачи (флаг 0/1 определяет наличие выдачи по заявке);
INDEX_SUM (Number) — сумма по продукту.
Задание 1. Сгруппируйте по месяцам количество заявок и сумму выдач, вычислите долю выдач.

Задание 2. Определите источник, через который пришло больше всего заявок.

Задание 3. Выделите количество заявок, которые являются ошибочными.

Критерии ошибки:

Статус заявки не проставляется или имеет значение «тестовая заявка».
Заявка была создана в августе 2018 года.
Присутствует индикатор выдачи, но при этом лид не был зарегистрирован в системе.
Задание 4. Каким будет результат выполнения запросов ниже, если в поле CREATE_DATE будет содержаться значение 30.11.2017?

select trunc(create_date, 'dd') , create_date from test_sql
select trunc(create_date, 'mm') , create_date from test_sql
select trunc(create_date, 'mm')+1, create_date from test_sql